In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg

from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)
sa_dask = dg.from_geopandas(sa, npartitions=4)


### pga data

In [7]:
%%time
pga = dd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')

geometry = [Point(xy) for xy in zip(pga.lon, pga.lat)]
pga['geometry'] = geometry
crs = "EPSG:2193"
pgaGeo= gpd.GeoDataFrame(pga.compute(), crs=crs, geometry=geometry)
# pga = dg.set_geometry(df, geometry=df[['pickup_longitude', 'pickup_latitude']],
#                      crs={'init' :'epsg:4326'})

pga_dask= dg.from_geopandas(pgaGeo, npartitions=4)
# del pgaGeo


TypeError: Column assignment doesn't support type list

# Merge SA data with PGA events (dask-geodataframe)

In [36]:
%%time
from spatialpandas import sjoin
pga_with_saData = sjoin(pga_dask, sa_dask)
# pga_with_saData = dg.sjoin(pgaGeo, sa_dask, op="within")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Error in sys.excepthook:
Traceback (most recent call last):
  File "/Users/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2047, in showtraceback
Traceback (most recent call last):
  File "/Users/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent ca

KeyboardInterrupt: 